In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()
from datetime import datetime

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "ynrec1yahs2!"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width':'160px'}), href="http://www.snhu.edu", target="_blank")),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.P("Created by Shamus Cerny", style={"color":"rgb(202, 20, 76)"})),
    html.Hr(),
    html.Div(
        
dcc.RadioItems(
    id='filter-type',
    options=[
        {'label': 'Water Rescue', 'value': 'Water'},
        {'label': 'Mountain Rescue', 'value': 'Mountain'},
        {'label': 'Disaster Rescue', 'value': 'Disaster'},
        {'label': 'Reset (Show All)', 'value': 'All'}
    ],
    value='All',
    labelStyle={'display': 'inline-block', 'margin': '5px 0'},
    inputStyle={'margin-right': '8px'}
),

    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         page_size = 30,
                         row_selectable = 'single',
                         sort_action = 'native',
                         selected_rows = [0]
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
     # Copy df to avoid modifying the original
    temp_df = df.copy()

    # Define filters
    if filter_type == 'Water':
        breeds = ['Labrador Retriever', 'Chesa Bay Retr', 'Newfoundland']
        sex = 'Intact Female'
        min_weeks, max_weeks = 26, 156

    elif filter_type == 'Mountain':
        breeds = ['German Shepherd', 'Alaskan Husky','Old English Sheepdog', 'Siberian Husky', 'Rottweiler']
        sex = 'Intact Male'
        min_weeks, max_weeks = 26, 156

    elif filter_type == 'Disaster':
        breeds = ['Doberman Pinsch', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
        sex = 'Intact Male'
        min_weeks, max_weeks = 20, 300

    else:
        # Default (no filter)
        filtered_df = temp_df
        columns = [{"name": i, "id": i} for i in filtered_df.columns]
        data = filtered_df.to_dict('records')
        return data

    # Apply filters
    filtered_df = temp_df[
        temp_df['breed'].str.contains('|'.join(breeds), case=False, na=False)&
        (temp_df['sex_upon_outcome'] == sex) &
        (temp_df['age_upon_outcome_in_weeks'].between(min_weeks, max_weeks))
    ]

    # Return updated table
    data = filtered_df.to_dict('records')
    return data

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Use filtered data if available, otherwise full dataset
    dff = pd.DataFrame(viewData) if viewData else df.copy()

    if dff.empty:
        return [html.P("No data to display")]

    # Count animals per breed
    breed_counts = dff['breed'].value_counts()

    # Create pie chart
    fig = px.pie(
        names=breed_counts.index,
        values=breed_counts.values,
        title='Preferred Animals',
        color_discrete_sequence=px.colors.qualitative.Set3
    )

    # Show label only if the slice is bigger than 3% (adjust as needed)
    fig.update_traces(
        textinfo='label+percent',
        textposition='inside',
        texttemplate=[
            f'%{{label}}<br>%{{percent:.1%}}' if v / breed_counts.sum() > 0.03 else ''
            for v in breed_counts.values
        ],
        hoverinfo='label+percent+value'
    )

    fig.update_layout(margin=dict(t=40, b=10, l=10, r=10))
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    if selected_rows is None:
        selected_rows = []
    return [{
        'if': { 'row_index': i },
        'backgroundColor': '#D2F3FF'
    } for i in selected_rows]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://sandrashannon-planetsoda-3000.codio.io/proxy/8050/
